In [1]:
import os
import requests
import argparse
import csv
import pathlib 
import time
from datetime import datetime
from bs4 import BeautifulSoup


In [2]:

# Función para obtener la lista de indices del Ibex 35 de la bolsa de Barcelona
# Tendremos una tabla donde la primera fila el nombre de la empresa y los índices
# El resto de filas contendrá los valores; el valor del primer elemento será una tupla
        
# Get table Page        
def getPageTable(UrlPage):        
    table = None
    try:
        # Data for post form    
        data = {'punto':'indice'}    
        page = requests.post(UrlPage,timeout=10)    
        soup = BeautifulSoup(page.text, "html.parser")
        table = soup.find('table', attrs={'id': 'ctl00_Contenido_tblAcciones'})                           
    except requests.exceptions.Timeout:
        print("Timeout a la càrrega de la web")        
    except:        
        print("Error inesperat")
    return table        

# Get Firm page and wait x seconds
def getFirmTablePage(UrlFirmPage):    
    table = None    
    try:
        t0 = time.time()
        # Data for post form            
        data = {'punto':'indice'}    
        page = requests.get(UrlFirmPage,timeout=10)                    
        soup = BeautifulSoup(page.text, "html.parser")                
        table = soup.find('table', attrs={'id': 'ctl00_Contenido_tblCapital'})                           
        # Wait time depending of response
        response_delay = time.time() - t0     
        time.sleep(10 * response_delay)
    except requests.exceptions.Timeout:
        print("-->Timeout a la càrrega de dada de la companyia")                
    except:                
        print("-->Error inesperat: " +  UrlFirmPage)
    return table

# Get the list of firm interesting values
def getFirmValues(UrlBase,firmPage):
    Url = UrlBase+firmPage        
    values = []
    table = getFirmTablePage(Url)        
    if (table is not None):                
        nroFila=0
        rowNum=0
        for row in table.find_all("tr"):
            if rowNum==1:
                colNum=0                
                for cell in row.find_all('td'):
                    if colNum==1:
                        value=[cell.text,""]              
                        values.append(value)                                
                        break;
                    colNum=colNum+1
            rowNum=rowNum+1       
    return values    
        

# Set List with table page values
def getIndexs(UrlBase,UrlRef):
    indexList = []
    UrlIndexPage = UrlBase+UrlRef
    table = getPageTable(UrlIndexPage)    
    # Extract table values and links
    rowList = []
    for tr in table.find_all("tr"):        
        colList = []               
        valuesFirm = []
        # Cells Header
        cellsHead = tr.findAll('th')                
        if ((cellsHead is not None) and (len(cellsHead) > 0)):
            for cellHead in cellsHead:        
                value = [cellHead.text.strip(),'']                  
                colList.append(value)                                                   
            colList.append(['Capitalización',''])
        else:
            # Cells value
            cells = tr.findAll('td')                        
            for cell in cells:
                href = ''
                infComp = []
                a= cell.find('a')                  
                if (a is not None):                 
                    href = a.get('href')                                
                    # Append / if needed
                    if href[0] !='/':
                        href = '/' + href                     
                    valuesFirm = getFirmValues(UrlBase,href) 
                    # print firm
                    print(cell.text)
                value = [cell.text.strip(),href]                                  
                colList.append(value)             
                
        # Extend list with firm information 
        colList.extend(valuesFirm)
        # Append to rowlist    
        rowList.append(colList)         
    return rowList 

def writecsv(table,filename):
    currentDir = os.path.abspath('')
    filePath = os.path.join(currentDir,filename)
    file = open(filePath,"w+")
    for row in table:
        for col in row:
            file.write(col[0]+";")
        file.write("\n")        
    

In [3]:
# indicar la ruta
urlbase = 'https://www.borsabcn.es'
urlRef= '/esp/aspx/Mercados/Precios.aspx?indice=ESI100000000&punto=indice'
indexList = getIndexs(urlbase,urlRef);
print(indexList)



ACCIONA
ACERINOX
ACS
AENA
ALMIRALL
AMADEUS
ARCELORMIT.
B.SANTANDER
BA.SABADELL
BANKIA
BANKINTER
BBVA
CAIXABANK
CELLNEX
CIE AUTOMOT.
ENAGAS
ENDESA
FERROVIAL
GRIFOLS CL.A
IAG
IBERDROLA
INDITEX
INDRA A
INM.COLONIAL
MAPFRE
MELIA HOTELS
MERLIN
NATURGY
PHARMA MAR
R.E.C.
REPSOL
SIEMENS GAME
SOLARIA
TELEFONICA
VISCOFAN
[[['Nombre', ''], ['Últ.', ''], ['% Dif.', ''], ['Máx.', ''], ['Mín.', ''], ['Volumen', ''], ['Efectivo (miles €)', ''], ['Fecha', ''], ['Hora', ''], ['Capitalización', '']], [['ACCIONA', '/esp/aspx/Empresas/FichaValor.aspx?ISIN=ES0125220311'], ['140,7000', ''], ['1,08', ''], ['141,4000', ''], ['138,5000', ''], ['125.694', ''], ['17.598,45', ''], ['26/03/2021', ''], ['Cierre', ''], ['7.718.331', '']], [['ACERINOX', '/esp/aspx/Empresas/FichaValor.aspx?ISIN=ES0132105018'], ['10,8000', ''], ['3,65', ''], ['10,8000', ''], ['10,4650', ''], ['782.046', ''], ['8.362,05', ''], ['26/03/2021', ''], ['Cierre', ''], ['2.921.899', '']], [['ACS', '/esp/aspx/Empresas/FichaValor.aspx?ISIN=ES016

In [6]:
writecsv(indexList,"ListaIndices.csv")